# LLM comprehensive summary

Anton Antonov   
[MathematicaForPrediction at WordPress](https://mathematicaforprediction.wordpress.com)   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
April-May 2025

In [ ]:
"Generated on {Date.today.Str}"

---

## Introduction

In this computational Markdown file we apply different LLM prompts in order to comprehensively (and effectively) summarize large texts.

**Remark:** This Markdown file is intended to serve as a template for the initial versions of comprehensive text analyses.

**Remark:** This Markdown template has corresponding notebooks versions: 
(i) [Wolfram Language notebook](https://community.wolfram.com/groups/-/m/t/3448842), 
(ii) [Raku-Jupyter notebook](),
(iii) [Python-Jupyter notebook]().

**Remark:** All remarks in italics are supposed to be removed.

---

## Setup

Load packages:

In [ ]:
use LLM::Functions;
use LLM::Prompts;
use Text::SubParsers;
use ML::FindTextualAnswer;
use Data::Importers;
use Data::Translators;
use WWW::MermaidInk;
use WWW::YouTube;
use Hash::Merge;
use Graph;

Define LLM access configurations:

In [ ]:
my $conf4o-mini = llm-configuration('ChatGPT', model => 'gpt-4o-mini', max-tokens => 8192, temperature => 0.5);
my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', max-tokens => 8192, temperature => 0.5);
my $conf-gemini-flash = llm-configuration('Gemini', model => 'gemini-2.0-flash', max-tokens => 8192, temperature => 0.5);

# Choose an LLM access configuration or specify your own
sink my $conf = $conf4o-mini;

Choose an output language:

In [ ]:
sink my $lang = 'English';

---

-----

## Ingestion

**Remark:** Chose whether to analyze a text from a file or to analyze the transcript of a YouTube video.

Ingest text from a file:

In [0]:
#my $txtFocus = slurp('');
#text-stats($txtFocus)

Ingest the transcript of a YouTube video:

In [ ]:
my $txtFocus = youtube-transcript("eIR_OjWWjtE", format => 'text');
text-stats($txtFocus)

**Remark:** The text ingested above is the transcript of the video ["Live CEOing Ep 886: Design Review of LLMGraph"](https://www.youtube.com/watch?v=ewU83vHwN8Y).

**Remark:** The transcript of a YouTube video can be obtained in several ways:

- Use the Raku package ["WWW::YouTube"](https://raku.land/zef:antononcube/WWW::YouTube)
- On macOS, download the audio track and use the program [hear](https://sveinbjorn.org/hear) 
- Use the Python package [“pytube”](https://pypi.org/project/pytube/) (or [“pytubefix”](https://pypi.org/project/pytubefix/)) 


---

## Summary

Summarize the text:

In [ ]:
#% markdown
llm-synthesize([llm-prompt("Summarize"), $txtFocus, llm-prompt('Translated')('English')], e => $conf)

---

## Tabulate topics

Extract and tabulate text topics:

In [ ]:
#% html
my $tblThemes = llm-synthesize(llm-prompt("ThemeTableJSON")($txtFocus, "article", 30), e => $conf, form => sub-parser('JSON'):drop);
$tblThemes ==> data-translation(field-names=><theme content>)

---

## Mind-map

### Graph

In [ ]:
my @edges = 
    |llm-synthesize([
        "Make a JSON array with the graph edges of a concise mind-map for the following text.",
        "Each edge is a dictionary with keys 'from' and 'to'.",
        "Make sure the graph is connected and represents a mind-map.",
        "TEXT START",
        $txtFocus,
        "TEXT END",
        llm-prompt("NothingElse")("JSON")
    ], 
    e=>$conf,
    form => sub-parser('JSON'):drop
    );

deduce-type(@edges)

In [ ]:
#% html
my $g = Graph.new(@edges);

$g.dot(engine => 'neato', background => '#1F1F1F'):svg

### Mermaid-JS

Generate Mermaid-JS code of a corresponding mind-map:

In [ ]:
#% markdown

my $mmdBigPicture = llm-synthesize([
    "Create a concise mermaid-js mind-map for the text.",
    "TEXT START",
    $txtFocus,
    "TEXT END",
    llm-prompt("NothingElse")("correct mermaid-js in $lang")
], e=>$conf);

---

## Sophisticated feedback

Give sophisticated feedback using different “idea hats”:

In [ ]:
#% markdown
my $sophFeed = llm-synthesize(llm-prompt("SophisticatedFeedback")($txtFocus, 'HTML', :!ideas), e => $conf);

$sophFeed.subst( / ^^ '```html'/, :g).subst(/ ^^ '```'/, :g)

---

## Specific questions

Get answers to specific questions (if any.)

In [ ]:
my $questions = q:to/END/;
What technology? What it is used for?"
END

In [ ]:
#% markdown
my $ans = llm-synthesize([$questions, $txtFocus], e => $conf);

#### Structured

**Remark:** Instead of using the simple workflow above, more structured question-answer response can be obtained 
using the function `find-textual-answer` of the package ["ML::FindTextualAnswer"](https://raku.land/zef:antononcube/ML::FindTextualAnswer).

In [ ]:
#% markdown
find-textual-answer(
        $txtFocus, 
        ["Who is talking?", "Which technology is discussed?", "What product(s) are discussed?", "Which versions?"], 
        finder => llm-evaluator($conf))

---

## Extracted wisdom or cynical insights

**Remark:** Choose one of the prompts 
[“ExtractArticleWisdom”](https://www.wolframcloud.com/obj/antononcube/DeployedResources/Prompt/ExtractArticleWisdom/) or 
[“FindPropagandaMessage”](https://www.wolframcloud.com/obj/antononcube/DeployedResources/Prompt/FindPropagandaMessage/).
(The latter tends to be more fun.)

In [ ]:
my $prompt = True ?? llm-prompt("ExtractArticleWisdom")() !! llm-prompt("FindPropagandaMessage");
text-stats($prompt)

In [ ]:
#% markdown

my $sumIdea = 
    llm-synthesize([
        $prompt,
        'TEXT START',
        $txtFocus,
        'TEXT END'
     ], e => $conf);

$sumIdea.subst(/ ^^ '#' /, '###', :g)